<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY = '09e0673129344dc7e4c9958fd482548f'
movie = tmdb.Movies(778)

In [3]:
movie.info()

{'adult': False,
 'backdrop_path': '/zF95knRgouUrBNbUh5oqRI5NIcE.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}],
 'homepage': '',
 'id': 778,
 'imdb_id': 'tt0046487',
 'original_language': 'fr',
 'original_title': 'Les vacances de Monsieur Hulot',
 'overview': 'Monsieur Hulot, Jacques Tati’s endearing clown, takes a holiday at a seaside resort, where his presence provokes one catastrophe after another. Tati’s masterpiece of gentle slapstick is a series of effortlessly well-choreographed sight gags involving dogs, boats, and firecrackers; it was the first entry in the Hulot series and the film that launched its maker to international stardom.',
 'popularity': 4.765439,
 'poster_path': '/r4F4tsU0Ajeh9ZYUkWOJSYmioj7.jpg',
 'production_companies': [{'id': 478, 'name': 'Cady'},
  {'id': 479, 'name': 'Discina'}],
 'production_countries': [{'iso_3166_1': 'FR', 'name': 'France'}],
 'release_date': '1953-02-26',
 'r

In [48]:
import http.client
import json

conn = http.client.HTTPSConnection("api.themoviedb.org")

payload = "{}"

conn.request("GET", "/3/movie/latest?language=en-US&api_key=09e0673129344dc7e4c9958fd482548f", payload)

res = conn.getresponse()
data = json.loads(res.read().decode('utf-8'))

In [ ]:
latest_id = 492957
data = pd.DataFrame(columns = data.keys())
links = pd.read_csv('links.csv')
for curr_film_id in tqdm(links.tmdbId.values):
    try:
        curr_movie = tmdb.Movies(curr_film_id)
        data.loc[len(data)] = pd.Series(curr_movie.info()).values
    except:
        pass

 22%|██▏       | 5965/27278 [1:12:26<4:18:48,  1.37it/s]

Что-то медленно скачиваются данные :(

In [4]:
data = pickle.load(open('fresh.pkl', 'rb'))

In [5]:
data = pd.DataFrame(data).T
data.index = np.arange(len(data))

In [6]:
data[:3]

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
5,False,/3EqYpbGCE9S5GddU2K4cYzP5UmI.jpg,"[80, 35]",5,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.52978,/eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg,1995-12-09,Four Rooms,False,6.5,578
6,False,/eXKgVIjLCFNpQkjVg1VpA8yM2GA.jpg,"[28, 53, 80]",6,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",6.11669,/lNXmgUrP6h1nD53gkFh4WDzT6RZ.jpg,1993-10-15,Judgment Night,False,6.4,84
8,False,/udvB86uyzJ6P9vmB83WfrCbnmnI.jpg,[99],8,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,1.6097,/8YyIjOAxwzD3fZMdmJrfiApod4l.jpg,2006-01-01,Life in Loops (A Megacities RMX),False,6.4,4


## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
overview = data.overview.values

Посчитаем tf и idf и оставим их в виде разреженных матриц, если потом надо будет - будем уже из них получать конкретные значения для фильмов

In [48]:
co = CountVectorizer()
counter = co.fit_transform(overview).astype(float)
tf = 0.5 * (counter.multiply(1. / counter.max(axis=1).todense()))
idf = np.log(tf.shape[0] / np.array(counter.sum(axis=0)))[0]

tf = tf.tocsr()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [196]:
bigram_vectorizer = CountVectorizer(ngram_range=(2,2))
bigram_counter = co.fit_transform(overview)
bigram_counter = bigram_counter.tocsr()

In [58]:
idf

array([  7.23474867,   4.8271108 ,  11.40913594, ...,  12.10228312,
        12.10228312,  12.10228312])

## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 99114.

In [32]:
# put your code here
links = pd.read_csv('links.csv')
links[links.movieId == 99114]

,movieId,imdbId,tmdbId
20124,99114,1853728,68718.0


Нашли индекс нашего фильма:

In [197]:
def count_tfidf(row_number):
    global tf
    global idf
    
    new_tf = np.array(tf[row_number].todense())[0] + 0.5
    return new_tf * idf

def count_bigram(row_number):
    global bigram_counter
    
    return np.array(bigram_counter[row_number].todense())[0]

def get_row_number_by_id(curr_id):
    return data[data.id == curr_id].index[0]

Выведем tfidf для данного фильма (кстати, это Джанго.Освобожденный)

In [64]:
row_number = data[data.id == 68718.0].index[0]
count_tfidf(row_number)[:10]

array([ 3.61737433,  2.4135554 ,  5.70456797,  6.05114156,  6.05114156,
        6.05114156,  5.35799438,  6.05114156,  6.05114156,  6.05114156])

## Part 4
Для каждого пользователя найдите топ жанров, которые нравятся пользователю (можно делать как на семинаре top 5, top 3 и top 2) и порекомендуйте таким пользователя фильмы близкие по текстовому описанию (см. пункт 2). Для оценки результатов пункта 3 используйте метрику HitRate и метрики ранжирования (N)DCG. Формулу для подсчета HitRate можно взять на семинаре,  (N)DCG: https://en.wikipedia.org/wiki/Discounted_cumulative_gain



Здесь я планировал, но не успел доделать, следующее:
- получаем любимые жанры пользователя
- получаем любимые фильмы пользователя

Затем для каждого пользователя отдельно:
- среди всех фильмов фильтруем те, которые относятся ко всем трем любимым жанрам 

(до этого момента доделано)
- выбираем из них на основе наших сгенерированных выше фичей те, которые ближе всего к любимым фильмам пользователя
- выводим их

In [67]:
# put your code here
raitings = pd.read_csv('ratings.csv')

In [141]:
from collections import Counter

def get_top_movies(top_size):
    return raitings.sort_values(['rating']).groupby(['userId']).tail([top_size]).sort_values(['userId'])

def genre_list(x):
    return list(set(np.hstack(x)))

def genre_list_len(x):
    return len(genre_list(x))

def popular_genres(x):
    return Counter(np.hstack(x)).most_common(3)

def convert_popular_genres(x):
    return np.array(x)[:,0]

In [133]:
genres_table = data[['id', 'genre_ids']]
genres_table.columns = ['movieId', 'genre_ids']

In [134]:
df_joined = pd.merge(raitings, genres_table, on="movieId", how="inner")
top_genres = df_joined.groupby('userId').agg({'genre_ids': [genre_list, genre_list_len, popular_genres]})

In [177]:
top_genres['favor_genres'] = top_genres.genre_ids.popular_genres.apply(convert_popular_genres)

In [190]:
top_genres.drop(top_genres.columns[0, 1, 2], axis=1, inplace=True)

Посчитали любимые жанры:

In [200]:
top_genres[:5]

,favor_genres
,
userId,
1,"[18, 35, 53]"
2,"[18, 53, 28]"
3,"[18, 53, 35]"
4,"[18, 35, 10749]"
5,"[18, 35, 53]"


In [193]:
top_films = get_top_movies(5)

Нашли любимые фильмы:

In [195]:
top_films[:10]

,userId,movieId,rating,timestamp
158,1,7153,5.0,1112484633
142,1,5952,5.0,1112484619
170,1,8507,5.0,1094786027
131,1,4993,5.0,1112484682
30,1,1196,4.5,1112484742
191,2,1210,5.0,974820598
180,2,260,5.0,974821014
192,2,1214,5.0,974821014
177,2,70,5.0,974820691
181,2,266,5.0,974820748


In [140]:
users = list(set(raitings.userId))
recomendations = []

Посчитаем число табличку с жанрами фильмов:

In [218]:
list_genres_table = pd.DataFrame(index=genres_table.movieId, 
                                 data=list(map(lambda x: '|'.join(np.array(x).astype(str)), genres_table.genre_ids.values)))
list_genres_table[:5]

,0
movieId,
5,80|35
6,28|53|80
8,99
9,18
12,16|10751


Здесь как раз планировалась функция нахождения релевантных фильмов, но из нее пока готова только часть с отвевом кандидатов по жанрам. Как видим, всегда хотя бы 300 фильмов, да есть, так что будет из чего выбрать

In [235]:
for curr_user in tqdm(users[:10]):
    curr_top_films = top_films[top_films.userId == curr_user]
    curr_top_genres = top_genres.loc[curr_user].favor_genres[0]
    pretenders = list_genres_table[list_genres_table[0].str.contains(str(curr_top_genres[0])) &
                                   list_genres_table[0].str.contains(str(curr_top_genres[1])) &
                                   list_genres_table[0].str.contains(str(curr_top_genres[2]))]
    

 10%|█         | 1/10 [00:00<00:03,  2.77it/s]

(308, 1)


 20%|██        | 2/10 [00:00<00:02,  2.71it/s]

(1149, 1)


 30%|███       | 3/10 [00:01<00:02,  2.75it/s]

...

(308, 1)


 90%|█████████ | 9/10 [00:03<00:00,  2.70it/s]

(2128, 1)


100%|██████████| 10/10 [00:03<00:00,  2.70it/s]

(2128, 1)
